# Idea
- Get first timestamp
- Resample Dataset to same timewindow
- Merge?

In [1]:
DEVICE_SOURCE = "Heqing_device2"

TIME_WINDOW = "40s"

In [2]:
from pathlib import Path

import numpy as np
import pandas as pd
import pickle

In [3]:
import sys
repo_base_path = Path("./../").resolve()
assert str(repo_base_path).endswith("csg_is"), f"{repo_base_path} is not a valid path to the CSG_IS repository" 

sys.path.append(str(repo_base_path))

from py_dataset import get_all_files_df
from py_dataset import read_in_files
from py_dataset import feature_plotting
from py_dataset import feature_selection
from py_dataset import net_feature_engineering
from py_dataset import entropy_feature_engineering
from py_dataset import sys_func

from py_dataset.classes import DataSource

In [4]:
feature_path = repo_base_path / "data_merg" /"top_260_features.pkl"

with open(feature_path, "rb") as f:
    SELECTED_FEATURES = pickle.load(f)

In [5]:
data_path = Path('/media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/')
assert data_path.exists()

df = get_all_files_df.main(data_path)
df.head(1)

txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device1/3_thetick_4h/collections.txt
txt file found, will drop /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/code&data/0_raw_collected_data/Heqing/device2/1_normal/note.txt
                                           file_path
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...
RangeIndex(start=0, stop=622, step=1)
filetype
csv    530
zip     92
Name: count, dtype: int64
                                           file_path filetype  filesize_bytes
0  /media/<User>/DC/MAP_CreationOfNewDatasetsForDFL/...      csv           49419
data_source
SYS_data        92
KERN_data       91
FLS_data        91
RES_data        91
block_data      91
network_data    91
entropy_data    75
Name: count, dtype: int64


,file_path,filetype,filesize_bytes,researcher_name,device_name,label,data_source,file_name,device
0,/media//DC/MAP_CreationOfNewDatasetsForDFL/...,csv,49419,Heqing,device1,1_normal,block_data,e4_5f_01_68_35_e6.csv,Heqing_device1


In [6]:
single_dev = df[df["device"] == DEVICE_SOURCE]
assert len(single_dev["device"].unique()) == 1
print(single_dev.shape)

df = single_dev

(133, 9)


### Get First TimeStamp

In [7]:
def prepare_files(row, resample_time_window):
    if row["filetype"] != "csv" and row["filetype"] != "zip":
        return None

    min_t, max_t, df_t = read_in_files.get_all_timestamps(row['file_path'], row["filetype"],
                                            resample_time_window=resample_time_window)

    row["min_t"] = min_t
    row["max_t"] = max_t

    return row

rows = []
for k, row in df.iterrows():
    x = prepare_files(row, resample_time_window=None)

    rows.append(x)

df_timestamps = pd.DataFrame(rows)

! Dropped rows in DataFrames: (2,)
time                                                time
cpu                                                  cpu
seconds                                          seconds
ioread                                            ioread
iowrite                                          iowrite
                                         ...            
rpm:rpm_return_int                    rpm:rpm_return_int
fib:fib_table_lookup                fib:fib_table_lookup
raw_syscalls:sys_enter            raw_syscalls:sys_enter
random:credit_entropy_bits    random:credit_entropy_bits
timestamp                                            NaN
Name: 318, Length: 118, dtype: object
time                                                time
cpu                                                  cpu
seconds                                          seconds
ioread                                            ioread
iowrite                                          iowrite
               

In [8]:
START = df_timestamps["min_t"].min()
f'Timestamps between: {df_timestamps["min_t"].min()} and {df_timestamps["max_t"].max()}'

'Timestamps between: 2023-11-09 15:18:45 and 2023-11-13 02:07:23'

### Resample files

In [9]:
def only_return_selected_columns(df: pd.DataFrame) -> pd.DataFrame:
    selected_cols = SELECTED_FEATURES & set(df.columns)
    return df[list(selected_cols)]

def create_window_sliced_depending_on_type(df: pd.DataFrame, start_timestamp: pd.Timestamp) -> pd.DataFrame:
    data_type = DataSource.from_columns(df.columns.tolist())
    print(f"Data type: {data_type}")

    match data_type:
        case (
            DataSource.FileSystem | DataSource.KernelEvents | DataSource.ResourceUsageEvents | DataSource.ResourceUsageEvents_RP4  | DataSource.BlockInputOutputEvents
        ):
            df = only_return_selected_columns(df)
            df = feature_plotting.test_transform_all_to_numeric_columns(df.copy())
            return df.resample(TIME_WINDOW, origin=start_timestamp).mean()
        case DataSource.NetRecords:
            mean_df = net_feature_engineering.create_mean_net_df_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            return only_return_selected_columns(mean_df)
        case DataSource.Entropy:
            if "entropy_file_count" not in SELECTED_FEATURES:
                return None
            return entropy_feature_engineering.resample_high_entropy_grouping_by_label_single_label(df, time_window=TIME_WINDOW, start=start_timestamp)
            
        case DataSource.SystemCalls:
            raise NotImplementedError("SystemCalls not implemented yet")
        

def check_if_keep_source(columns):
    data_type = DataSource.from_columns(columns.tolist())
    if data_type in [DataSource.SystemCalls, DataSource.Entropy, DataSource.NetRecords]:
        return True

    chosen_cols_from_source = data_type.value & SELECTED_FEATURES

    return any(chosen_cols_from_source)

In [10]:

df_mean_collection: list[pd.DataFrame] = []

for source, group_df in df.groupby(["data_source"]):
    dfs_same_source = []

    checked_source = False
    for k, row in group_df.iterrows():
        if row["filetype"] != "csv":
            print(f"Skipping {row['file_path']}")
            continue


        df_t = read_in_files.get_all_columns(row['file_path'], row["filetype"])
        if not checked_source:
            if not check_if_keep_source(df_t.columns):
                print(f"Skipping {source}")
                break
            checked_source = True

        try:
            df_mean = create_window_sliced_depending_on_type(df_t, start_timestamp=START)
        except Exception as ex:
            print(f"Problem in file {row['file_path']}")
            raise ex
        
        if df_mean is None:
            print(f"Skipping {row['file_path']}")
            continue
        
        df_mean = df_mean.copy()
        print(f"Shape: {df_mean.shape}")
        df_mean["label"] = row["label"]
        df_mean["device"] = row["device"]
        df_mean["data_source"] = row["data_source"]

        # print(f"Start and End of Index Timestamp: {df_mean.index[0]} - {df_mean.index[-1]}")
        dfs_same_source.append(df_mean)

    if len(dfs_same_source) == 0:
        print(f"Skipping {source}")
        continue
    same_df = pd.concat(dfs_same_source, axis=0)
    print(f"same_df shape: {same_df.shape}")
    df_mean_collection.append(same_df)
    # break

len(df_mean_collection)

Data type: DataSource.FileSystem
Shape: (148, 54)
Data type: DataSource.FileSystem
Shape: (236, 54)
Data type: DataSource.FileSystem
Shape: (136, 54)
Data type: DataSource.FileSystem
Shape: (95, 54)
Data type: DataSource.FileSystem
Shape: (136, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (180, 54)
Data type: DataSource.FileSystem
Shape: (180, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (180, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (180, 54)
Data type: DataSource.FileSystem
Shape: (182, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
Data type: DataSource.FileSystem
Shape: (181, 54)
same_df shape: (3282, 57)
Data type: DataSource.Ker

KeyboardInterrupt: 

#### Remove Dublicate Columns and Concatenate

=> some dont work => if Index error use merge manually

In [ ]:
try:
    pd.concat(df_mean_collection, axis=1)
except Exception as ex:
    print(ex)

Reindexing only valid with uniquely valued Index objects


In [ ]:
from collections import defaultdict

cols = defaultdict(int)
for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        cols[k] +=1

print("Dublicates:", [k for k, v in cols.items() if v > 1])

to_ignore = ["data_source", "label", "device"]

infos = pd.DataFrame(columns=to_ignore)

for dfs in df_mean_collection:
    for k in dfs.columns.tolist():
        if k in to_ignore:
            continue

        if cols[k] > 1:
            print(k)

            dfs.rename(columns={k: f'{k}_{dfs["data_source"].iloc[0]}'}, inplace=True)

    assert dfs["device"].nunique() == 1
    assert dfs["label"].nunique() == 9
    assert dfs["data_source"].nunique() == 1
    
    infos = pd.concat([infos,dfs[to_ignore]], axis=0)

    # for t,row in dfs.iterrows():
        # infos.append({"device": row["device"], "label": row["label"], "data_source": row["data_source"], "timestamp": t})
    
    dfs.drop(columns=to_ignore, inplace=True)

Dublicates: ['writeback:writeback_dirty_page', 'ext4:ext4_unlink_enter', 'filemap:mm_filemap_delete_from_page_cache', 'ext4:ext4_journal_start', 'writeback:sb_clear_inode_writeback', 'writeback:global_dirty_state', 'jbd2:jbd2_checkpoint_stats', 'writeback:writeback_mark_inode_dirty', 'ext4:ext4_es_lookup_extent_enter', 'jbd2:jbd2_start_commit', 'block:block_dirty_buffer', 'ext4:ext4_da_update_reserve_space', 'ext4:ext4_free_inode', 'block:block_bio_remap', 'block:block_touch_buffer', 'block:block_getrq', 'filemap:mm_filemap_add_to_page_cache', 'writeback:writeback_dirty_inode', 'block:block_bio_backmerge', 'jbd2:jbd2_handle_stats', 'jbd2:jbd2_handle_start', 'ext4:ext4_writepages_result', 'writeback:writeback_dirty_inode_enqueue', 'seconds', 'ext4:ext4_releasepage', 'block:block_unplug', 'label', 'device', 'data_source', 'kmem:kfree', 'kmem:mm_page_alloc_zone_locked', 'kmem:kmem_cache_alloc', 'fib:fib_table_lookup', 'gpio:gpio_value', 'tcp:tcp_probe', 'cpu-migrations', 'sched:sched_proc

/tmp/ipykernel_56296/3047974697.py:28: FutureWarning: The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
  infos = pd.concat([infos,dfs[to_ignore]], axis=0)


In [ ]:
df_all = pd.concat(df_mean_collection[:-1], axis=1).merge(df_mean_collection[-1], how='outer', left_index=True, right_index=True)
df_all.sample(2)

,writeback:writeback_dirty_page_FLS_data,ext4:ext4_ext_remove_space_done,writeback:writeback_dirty_inode_start,jbd2:jbd2_commit_logging,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_es_lookup_extent_exit,ext4:ext4_ext_remove_space,ext4:ext4_da_write_end,ext4:ext4_truncate_exit,filemap:mm_filemap_delete_from_page_cache_FLS_data,...,armv7_cortex_a15/bus_cycles/,iowrite,page-faults_RES_data,ext4:ext4_releasepage_RES_data,pkt_out,write_kbs,write_ops,util,(OLD) AverageLength,(OLD) VarianceLength
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-12 12:08:05,9.0,6.0,2969.0,2.0,1.0,834.0,6.0,36.0,6.0,8.0,...,8.283584e+09,57.333333,34249.000000,1060.000000,7449.333333,3.700,166.056667,27.780,1090.279446,413855.896266
2023-11-10 12:12:05,7.0,0.0,1308.0,2.0,0.0,86.0,0.0,0.0,0.0,16514.0,...,7.271517e+09,12.666667,21612.333333,0.666667,9772.000000,1.175,5.125000,8.135,1169.818253,357821.466942


In [ ]:
df_all = df_all.merge(infos["label"], how='outer', left_index=True, right_index=True)

In [ ]:
df_all.shape, len(SELECTED_FEATURES)

((17512, 205), 260)

In [ ]:
df_all

,writeback:writeback_dirty_page_FLS_data,ext4:ext4_ext_remove_space_done,writeback:writeback_dirty_inode_start,jbd2:jbd2_commit_logging,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_es_lookup_extent_exit,ext4:ext4_ext_remove_space,ext4:ext4_da_write_end,ext4:ext4_truncate_exit,filemap:mm_filemap_delete_from_page_cache_FLS_data,...,iowrite,page-faults_RES_data,ext4:ext4_releasepage_RES_data,pkt_out,write_kbs,write_ops,util,(OLD) AverageLength,(OLD) VarianceLength,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,5.5,19358.5,0.00,8657.00,0.475,3.6,9.8225,1194.224582,332853.277927,1_normal
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,5.5,19358.5,0.00,8657.00,0.475,3.6,9.8225,1194.224582,332853.277927,1_normal
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,5.5,19358.5,0.00,8657.00,0.475,3.6,9.8225,1194.224582,332853.277927,1_normal
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,5.5,19358.5,0.00,8657.00,0.475,3.6,9.8225,1194.224582,332853.277927,1_normal
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,5.5,19358.5,0.00,8657.00,0.475,3.6,9.8225,1194.224582,332853.277927,1_normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-11-13 02:06:45,0.0,0.0,1306.0,1.0,0.0,6.0,0.0,0.0,0.0,8711.0,...,3.5,9171.0,0.25,5264.25,0.325,2.6,8.3325,999.731343,467462.369501,9_xmrig
2023-11-13 02:06:45,0.0,0.0,1306.0,1.0,0.0,6.0,0.0,0.0,0.0,8711.0,...,3.5,9171.0,0.25,5264.25,0.325,2.6,8.3325,999.731343,467462.369501,9_xmrig
2023-11-13 02:06:45,0.0,0.0,1306.0,1.0,0.0,6.0,0.0,0.0,0.0,8711.0,...,3.5,9171.0,0.25,5264.25,0.325,2.6,8.3325,999.731343,467462.369501,9_xmrig


# Add Sys Call Data

In [ ]:
def read_in_preprocessed_Heqing_device2():
    file_path = Path('/media/<User>/DC/IS_Data_Exploration_and_Feature_Engineering_for_an_IoT_Device_Behavior_Fingerprinting_Dataset/sys_system_calls_Heqing_device2/merged_data.npz')
    assert file_path.exists(), 'Data not found'

    with np.load(file_path, allow_pickle=True) as npz:
        print(npz.files)
        X = npz['arr_0']
    print(X.shape)

    vectorizer = sys_func.create_CountVectorizer()
    cols = vectorizer.get_feature_names_out()
    cols = np.append(cols, [['timestamp'], ['label']])
    print (cols.shape)
    df = pd.DataFrame(X, columns=cols)
    # df.set_index('timestamp', inplace=True)
    read_in_files._clean_and_set_index_timestamp(df)

    label_encoder = sys_func.create_label_encoder_for_labels()
    df['label'] = label_encoder.inverse_transform(df['label'].values.astype(int))

    vectors = df.copy()
    print(vectors.shape)
    
    return vectors

sys_df = read_in_preprocessed_Heqing_device2()
# sys_df = only_return_selected_columns(sys_df)
# sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

['arr_0']
(11428, 162)
Loaded set: ('accept', 'accept4', 'access', 'add_key', 'adjtimex', 'bind', 'brk', 'capget', 'capset', 'chdir', 'chmod', 'chown32', 'clock_gettime', 'clone', 'close', 'connect', 'dup', 'dup2', 'dup3', 'epoll_create1', 'epoll_ctl', 'epoll_wait', 'eventfd2', 'execve', 'exit', 'exit_group', 'faccessat', 'fadvise64_64', 'fallocate', 'fchdir', 'fchmod', 'fchmodat', 'fchown32', 'fcntl64', 'fgetxattr', 'flistxattr', 'flock', 'fsetxattr', 'fstat64', 'fstatat64', 'fstatfs64', 'fsync', 'ftruncate64', 'futex', 'getcwd', 'getdents64', 'getegid32', 'geteuid32', 'getgid32', 'getgroups32', 'getpeername', 'getpgid', 'getpgrp', 'getpid', 'getppid', 'getpriority', 'getrandom', 'getresgid32', 'getresuid32', 'getsid', 'getsockname', 'getsockopt', 'gettid', 'gettimeofday', 'getuid32', 'getxattr', 'inotify_add_watch', 'ioctl', 'ioprio_get', 'ioprio_set', 'kcmp', 'keyctl', 'kill', 'lchown32', 'lgetxattr', 'llseek', 'lseek', 'lstat64', 'madvise', 'mkdir', 'mknod', 'mmap2', 'mount', 'mpro

In [ ]:
sys_df.head(2)

,accept,accept4,access,add_key,adjtimex,bind,brk,capget,capset,chdir,...,umount2,uname,unlink,unlinkat,utimensat,wait4,waitid,write,writev,label
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.0,0.000008,0.000770,0.0,0.0,0.00002,0.001104,0.000037,0.00001,0.000006,...,0.0,0.000186,0.000006,0.0,0.0,0.000305,0.000023,0.028951,0.000006,1_normal
2023-11-09 15:18:55,0.0,0.000023,0.000481,0.0,0.0,0.00000,0.000726,0.000017,0.00000,0.000005,...,0.0,0.000176,0.000008,0.0,0.0,0.000231,0.000020,0.029970,0.000005,1_normal


In [ ]:
sys_df = only_return_selected_columns(sys_df)
sys_df_mean = sys_df.resample(TIME_WINDOW, origin=START).mean()

In [ ]:
sys_df_mean.head(2)

,poll,epoll_wait,faccessat,pause,unlink,getsockopt,set_robust_list,prctl,recv,llseek,...,madvise,socket,connect,pipe,setitimer,accept4,send,mmap2,ftruncate64,clock_gettime
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,0.000232,0.000595,0.000000,0.000003,0.000006,0.000773,0.000064,0.000025,0.000023,0.036762,...,8.912445e-07,0.000750,0.000687,0.000117,0.000005,0.000018,0.000009,0.004991,0.000044,0.027907
2023-11-09 15:19:25,0.000489,0.000670,0.000002,0.000003,0.000010,0.000893,0.000086,0.000025,0.000032,0.041900,...,2.412710e-06,0.000836,0.000745,0.000174,0.000007,0.000022,0.000014,0.007471,0.000048,0.034694


In [ ]:
all_df1 = df_all.merge(sys_df_mean, how='outer', left_index=True, right_index=True)
print(f"all_df1 shape: {all_df1.shape}")

all_df1 shape: (21673, 306)


In [ ]:
all_df1.head(4)

,writeback:writeback_dirty_page_FLS_data,ext4:ext4_ext_remove_space_done,writeback:writeback_dirty_inode_start,jbd2:jbd2_commit_logging,ext4:ext4_unlink_enter_FLS_data,ext4:ext4_es_lookup_extent_exit,ext4:ext4_ext_remove_space,ext4:ext4_da_write_end,ext4:ext4_truncate_exit,filemap:mm_filemap_delete_from_page_cache_FLS_data,...,madvise,socket,connect,pipe,setitimer,accept4,send,mmap2,ftruncate64,clock_gettime
timestamp,,,,,,,,,,,,,,,,,,,,,
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,8.912445e-07,0.00075,0.000687,0.000117,0.000005,0.000018,0.000009,0.004991,0.000044,0.027907
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,8.912445e-07,0.00075,0.000687,0.000117,0.000005,0.000018,0.000009,0.004991,0.000044,0.027907
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,8.912445e-07,0.00075,0.000687,0.000117,0.000005,0.000018,0.000009,0.004991,0.000044,0.027907
2023-11-09 15:18:45,3.0,0.0,1264.5,1.0,0.0,487.0,0.0,0.0,0.0,1.5,...,8.912445e-07,0.00075,0.000687,0.000117,0.000005,0.000018,0.000009,0.004991,0.000044,0.027907


In [ ]:
spath = repo_base_path / "training" / "data" / f"all_df_{TIME_WINDOW}_{DEVICE_SOURCE}.csv"

all_df1.to_csv(str(spath))